<a href="https://colab.research.google.com/github/newacc11/chat-bot/blob/main/%D0%A5%D0%B0%D0%BA_%D0%BE%D1%82%D0%BA%D1%80%D1%8B%D1%82%D0%B8%D0%B5_%D1%80%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D1%80%D0%B5%D1%87%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pymorphy2

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 8.2MB 5.8MB/s 


In [3]:
import pandas as pd
import re
import string
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
import pymorphy2
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
df = pd.read_excel('/content/drive/My Drive/Мусор + попытки (искать и в других папках)/База знаний.xlsx')

In [30]:
df.head()

,Unnamed: 0,Запрос,Уточнение запроса,Тип ДБО,Вопрос,Ссылка на видео,Ссылка на вопрос в FAQ,Где используется на сайте,Шаг 1,Шаг 2,Шаг 3,Шаг 4,Шаг 5,Шаг 6,Шаг 7,Шаг 8,Шаг 9,Шаг 10,Шаг 11,Шаг 12,Шаг 13,Шаг 14
0,2.0,Карты,Активация,ИБ,Как активировать карту,NaN,NaN,https://www.open.ru/credit_cards/faq#1056\n \n...,На вкладке «Мои продукты» в разделе «Карты» вы...,Введите предпоследние 4 цифры номера карты — с...,Придумайте и повторно введите PIN-код. Он не д...,Нажмите «Активировать»,Введите SMS-код для подтверждения операции,Ваша карта активирована,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,Карты,Баланс,ИБ,Как узнать баланс карты,https://www.youtube.com/watch?v=M-C95qRN15Y&fe...,NaN,https://www.open.ru/faq#1096\n \n https://www....,На вкладке «Мои продукты» в разделе «Карты» вы...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7.0,Карты,История операций,ИБ,Как посмотреть историю операций по карте,NaN,NaN,NaN,Перейдите в раздел «История операций». На экра...,Выберите нужную карту,"Выберите период, тип операций и категории",Ниже появятся операции по карте по выбранным в...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9.0,Карты,Справки и выписки,ИБ,Как получить выписку по карте с синей печатью ...,https://www.youtube.com/watch?v=M-C95qRN15Y&fe...,NaN,NaN,Перейдите в раздел «История операций». На экра...,Выберите нужную карту,"Выберите период, тип (»Все», «Пополнения» или ...",Выберите формат выписки в поле «Получить выпис...,Выписку в формате PDF вы можете скачать или от...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11.0,Карты,Реквизиты,ИБ,Как посмотреть реквизиты счета карты,NaN,NaN,NaN,На вкладке «Мои продукты» найдите нужную карту...,Перейдите в раздел «Счет»,В открывшемся окне отобразятся реквизиты счета...,Реквизиты можно отправить на e-mail. Для этого...,ОЙ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df = df.iloc[:, 1:]

In [32]:
df

,Запрос,Уточнение запроса,Тип ДБО,Вопрос,Ссылка на видео,Ссылка на вопрос в FAQ,Где используется на сайте,Шаг 1,Шаг 2,Шаг 3,Шаг 4,Шаг 5,Шаг 6,Шаг 7,Шаг 8,Шаг 9,Шаг 10,Шаг 11,Шаг 12,Шаг 13,Шаг 14
0,Карты,Активация,ИБ,Как активировать карту,NaN,NaN,https://www.open.ru/credit_cards/faq#1056\n \n...,На вкладке «Мои продукты» в разделе «Карты» вы...,Введите предпоследние 4 цифры номера карты — с...,Придумайте и повторно введите PIN-код. Он не д...,Нажмите «Активировать»,Введите SMS-код для подтверждения операции,Ваша карта активирована,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Карты,Баланс,ИБ,Как узнать баланс карты,https://www.youtube.com/watch?v=M-C95qRN15Y&fe...,NaN,https://www.open.ru/faq#1096\n \n https://www....,На вкладке «Мои продукты» в разделе «Карты» вы...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Карты,История операций,ИБ,Как посмотреть историю операций по карте,NaN,NaN,NaN,Перейдите в раздел «История операций». На экра...,Выберите нужную карту,"Выберите период, тип операций и категории",Ниже появятся операции по карте по выбранным в...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Карты,Справки и выписки,ИБ,Как получить выписку по карте с синей печатью ...,https://www.youtube.com/watch?v=M-C95qRN15Y&fe...,NaN,NaN,Перейдите в раздел «История операций». На экра...,Выберите нужную карту,"Выберите период, тип (»Все», «Пополнения» или ...",Выберите формат выписки в поле «Получить выпис...,Выписку в формате PDF вы можете скачать или от...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Карты,Реквизиты,ИБ,Как посмотреть реквизиты счета карты,NaN,NaN,NaN,На вкладке «Мои продукты» найдите нужную карту...,Перейдите в раздел «Счет»,В открывшемся окне отобразятся реквизиты счета...,Реквизиты можно отправить на e-mail. Для этого...,ОЙ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df = df.loc[:288, :]

In [34]:
df

,Запрос,Уточнение запроса,Тип ДБО,Вопрос,Ссылка на видео,Ссылка на вопрос в FAQ,Где используется на сайте,Шаг 1,Шаг 2,Шаг 3,Шаг 4,Шаг 5,Шаг 6,Шаг 7,Шаг 8,Шаг 9,Шаг 10,Шаг 11,Шаг 12,Шаг 13,Шаг 14
0,Карты,Активация,ИБ,Как активировать карту,NaN,NaN,https://www.open.ru/credit_cards/faq#1056\n \n...,На вкладке «Мои продукты» в разделе «Карты» вы...,Введите предпоследние 4 цифры номера карты — с...,Придумайте и повторно введите PIN-код. Он не д...,Нажмите «Активировать»,Введите SMS-код для подтверждения операции,Ваша карта активирована,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Карты,Баланс,ИБ,Как узнать баланс карты,https://www.youtube.com/watch?v=M-C95qRN15Y&fe...,NaN,https://www.open.ru/faq#1096\n \n https://www....,На вкладке «Мои продукты» в разделе «Карты» вы...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Карты,История операций,ИБ,Как посмотреть историю операций по карте,NaN,NaN,NaN,Перейдите в раздел «История операций». На экра...,Выберите нужную карту,"Выберите период, тип операций и категории",Ниже появятся операции по карте по выбранным в...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Карты,Справки и выписки,ИБ,Как получить выписку по карте с синей печатью ...,https://www.youtube.com/watch?v=M-C95qRN15Y&fe...,NaN,NaN,Перейдите в раздел «История операций». На экра...,Выберите нужную карту,"Выберите период, тип (»Все», «Пополнения» или ...",Выберите формат выписки в поле «Получить выпис...,Выписку в формате PDF вы можете скачать или от...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Карты,Реквизиты,ИБ,Как посмотреть реквизиты счета карты,NaN,NaN,NaN,На вкладке «Мои продукты» найдите нужную карту...,Перейдите в раздел «Счет»,В открывшемся окне отобразятся реквизиты счета...,Реквизиты можно отправить на e-mail. Для этого...,ОЙ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Накопительные счета,Повышение ставки,ИБ,Как повысить ставку по накопительному счету «М...,NaN,https://www.open.ru/deposits/savings_account/f...,https://www.open.ru/deposits/savings_account/faq,Перейдите на экран программы лояльности МАКС: ...,Нажмите «Потратить бонусы».,Нажмите «Повысить ставку по счету».,"Выберите накопительный счет, для которого хоти...","Выберите, на какой срок повысить ставку: на од...",Перетяните подходящий вариант повышения ставки...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
285,Карты,Аналитика финансов,ИБ,Как посмотреть аналитику финансов по карте,NaN,https://www.open.ru/faq#1829,https://www.open.ru/faq,Перейдите на вкладку «История операций».,Выберите карту или несколько карт в одной валю...,Диаграмма с аналитикой финансов отобразится в ...,Аналитику финансов можно просматривать в разны...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286,Календарь платежей,NaN,ИБ,Как посмотреть календарь платежей,NaN,NaN,NaN,Перейдите на вкладку «Мои продукты» или «Плате...,Перейдите в раздел «Календарь платежей» и нажм...,Справа от календаря отобразится список всех пл...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287,Виртуальная карта,Дополнительная карта,ИБ,Как оформить дополнительную виртуальную карту,NaN,NaN,NaN,На экране «Мои продукты» в разделе «Карты» выб...,Выберите «Дополнительная виртуальная карта».,"Выберите платежную систему, ознакомьтесь с тар...",Введите SMS-код для подтверждения выпуска карты.,Карта будет выпущена через несколько минут и п...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df.loc[(df['Где используется на сайте'].isna() == False) & (df['Ссылка на вопрос в FAQ'].isna() == False)]

,Запрос,Уточнение запроса,Тип ДБО,Вопрос,Ссылка на видео,Ссылка на вопрос в FAQ,Где используется на сайте,Шаг 1,Шаг 2,Шаг 3,Шаг 4,Шаг 5,Шаг 6,Шаг 7,Шаг 8,Шаг 9,Шаг 10,Шаг 11,Шаг 12,Шаг 13,Шаг 14
14,Программа лояльности,Баланс,ИБ,Как узнать баланс бонусных рублей,NaN,https://www.open.ru/faq#1585,https://www.open.ru/faq#1585,На вкладке «Мои продукты» в разделе «Карты» вы...,Ниже отобразится баланс бонусных рублей,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,Программа лояльности,Начисления,ИБ,"Как узнать, сколько бонусов будет начислено за...",NaN,https://www.open.ru/faq#1586,https://www.open.ru/faq#1586,Нажмите на баланс бонусных рублей в верхней ча...,В новом окне отобразится количество бонусов к ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Программа лояльности,Условия,ИБ,Как узнать текущую программу лояльности,NaN,https://www.open.ru/faq#1588,https://www.open.ru/faq#1588,Нажмите на баланс бонусных рублей в верхней ча...,На открывшейся странице отобразятся текущие ус...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,Программа лояльности,Выполнение условий,ИБ,Как посмотреть статус выполнения условий по пр...,NaN,https://www.open.ru/faq#1587,https://www.open.ru/faq#1587,Нажмите на баланс бонусных рублей в верхней ча...,На открывшейся странице отобразится выполнение...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Программа лояльности,Выбор условий,ИБ,Как выбрать или изменить условия программы лоя...,NaN,https://www.open.ru/faq#1581,https://www.open.ru/faq#1581,Нажмите на баланс бонусных рублей в верхней ча...,На открывшейся странице нажмите на «Выбор усло...,Слева выберите подходящие условия программы ло...,Нажмите «Активировать»,В открывшемся окне отобразится дата перехода н...,Новая условия программы лояльности начнут дейс...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,Карты,Кредитные карты,ИБ,Как оформить кредитную карту,NaN,https://www.open.ru/credit_cards/faq#1048,https://www.open.ru/credit_cards/faq#1048,На экране «Мои продукты» в разделе «Карты» наж...,Выберите «Кредитные» и нажмите «Продолжить».,"Введите SMS-код, подтвердите согласие на обраб...",Проверьте и введите недостающие данные и нажми...,Введите дополнительные данные: доход и способ ...,Дождитесь предварительного одобрения карты.,"Если заявка предварительно одобрена, укажите р...","Выберите способ доставки карты, проверьте имя ...",Укажите информацию о дополнительном источнике ...,Дождитесь SMS с решением банка. Если заявка на...,NaN,NaN,NaN,NaN
267,Кредиты,Справки и выписки,ИБ,Как получить справку о выплаченных процентах п...,NaN,https://www.open.ru/credits/faq#967,https://www.open.ru/credits/faq#967,На вкладке «Мои продукты» в разделе «Кредиты» ...,Нажмите «Справки».,Нажмите «О выплаченных процентах».,"Выберите, за какой период оформить справку: за...","Нажмите «Скачать», чтобы сохранить справку, ил...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,Помощь,Чат,ИБ,Как написать в службу поддержки в интернет-банке,NaN,https://www.open.ru/ipoteka/faq#921,https://www.open.ru/ipoteka/faq,Нажмите на голубой значок сообщения в нижнем п...,Введите свой вопрос в текстовое поле,"Нажмите на голубую стрелку, чтобы отправить со...",Ожидайте ответа службы поддержки.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,Накопительные счета,Повышение ставки,ИБ,Как повысить ставку по накопительному счету «М...,NaN,https://www.open.ru/deposits/savings_account/f...,https://www.open.ru/deposits/savings_account/faq,Перейдите на экран программы лояльности МАКС: ...,Нажмите «Потратить бонусы».,Нажмите «Повысить ставку по счету».,"Выберите накопительный счет, для которого хоти...","Выберите, на какой срок повысить ставку: на од...",Перетяните подходящий вариант повышения ставки...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
len(df) - df['Ссылка на видео'].isna().sum()

4

In [37]:
len(df) - df['Ссылка на вопрос в FAQ'].isna().sum()

97

In [38]:
len(df) - df['Где используется на сайте'].isna().sum()

72

In [39]:
len(df)

289

In [40]:
df.index = range(len(df))

In [42]:
df['Вопрос_1'] = range(len(df))
for i in range(len(df['Вопрос'])):
  df['Вопрос_1'][i] = df['Вопрос'][i].lower()
  df['Вопрос_1'][i] = re.sub(r'\d+', '', df['Вопрос_1'][i])
  df['Вопрос_1'][i] = df['Вопрос_1'][i].translate(str.maketrans('', '', string.punctuation))
  df['Вопрос_1'][i] = df['Вопрос_1'][i].strip()
  df['Вопрос_1'][i] = word_tokenize(df['Вопрос_1'][i])
  stop_words = set(stopwords.words('russian'))
  df['Вопрос_1'][i] = [i for i in df['Вопрос_1'][i] if not i in stop_words]
  l = []
  for word in df['Вопрос_1'][i]:
    l.append(morph.parse(word)[0].normal_form)
  df['Вопрос_1'][i] = ' '.join(l)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.o

In [43]:
df['Вопрос_1']

0                                     активировать карта
1                                    узнать баланс карта
2                      посмотреть история операция карта
3      получить выписка карта синь печать банк факсимиле
4                         посмотреть реквизит счёт карта
                             ...                        
284    повысить ставка накопительный счёт « копилка »...
285                    посмотреть аналитик финансы карта
286                          посмотреть календарь платёж
287            оформить дополнительный виртуальный карта
288        оформить комплексный решение вклад инвестиция
Name: Вопрос_1, Length: 289, dtype: object

In [66]:
input_str = input()
input_str = input_str.lower()
input_str = re.sub(r'\d+', '', input_str)
input_str = input_str.translate(str.maketrans('', '', string.punctuation))
input_str = input_str.strip()
input_str = word_tokenize(input_str)
stop_words = set(stopwords.words('russian'))
input_str = [i for i in input_str if not i in stop_words]
morph = pymorphy2.MorphAnalyzer()
inf_words = []
for word in input_str:
  inf_words.append(morph.parse(word)[0].normal_form)
input_str = ' '.join(inf_words)

Как правильно и быс38тро ак#тивиров%ать Карту 3892


## Добавить стем перед лемом и анализ частей речи


In [67]:
from gensim import corpora, models, similarities
import jieba
texts = df['Вопрос_1']
keyword = input_str
texts = [jieba.lcut(text) for text in texts]
dictionary = corpora.Dictionary(texts)
feature_cnt = len(dictionary.token2id)
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus) 
kw_vector = dictionary.doc2bow(jieba.lcut(keyword))
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)
sim = index[tfidf[kw_vector]]
choice = {}
for i in range(len(sim)):
    choice[i] = sim[i]
choice = {k: v for k, v in sorted(choice.items(), key=lambda item: item[0])}

In [68]:
import operator
choice = sorted(choice.items(), key=operator.itemgetter(1), reverse=True)

In [69]:
choice

[(255, 0.8128647),
 (253, 0.7564411),
 (0, 0.7205862),
 (196, 0.70490056),
 (7, 0.67862046),
 (247, 0.6689127),
 (6, 0.6675709),
 (130, 0.66645825),
 (244, 0.6661406),
 (229, 0.62447643),
 (44, 0.5986229),
 (256, 0.5972203),
 (161, 0.59298015),
 (207, 0.58635956),
 (125, 0.5851064),
 (223, 0.58312964),
 (19, 0.58240587),
 (23, 0.5762301),
 (134, 0.55734164),
 (50, 0.5570484),
 (8, 0.54834527),
 (151, 0.54330325),
 (124, 0.53885055),
 (147, 0.53882366),
 (222, 0.5355302),
 (257, 0.5330073),
 (14, 0.5325474),
 (280, 0.53236824),
 (18, 0.5266882),
 (140, 0.52191025),
 (3, 0.5202586),
 (55, 0.5197092),
 (42, 0.51501244),
 (132, 0.511759),
 (252, 0.5057723),
 (254, 0.5057186),
 (284, 0.50297636),
 (60, 0.48999363),
 (34, 0.48929903),
 (43, 0.48635375),
 (107, 0.48456603),
 (188, 0.4843559),
 (93, 0.47721806),
 (135, 0.4771166),
 (22, 0.47571),
 (24, 0.4695662),
 (170, 0.46305197),
 (1, 0.4624397),
 (47, 0.4596735),
 (237, 0.45707566),
 (96, 0.45317534),
 (149, 0.44753343),
 (186, 0.44618148

In [62]:
choice[:3][0][0]

0

In [70]:
for i in range(len(choice[:3])):
  print(df['Вопрос'][choice[i][0]])

Как скрыть баланс вклада
Как скрыть баланс карты
Как активировать карту


In [71]:
df.to_csv('')

,Запрос,Уточнение запроса,Тип ДБО,Вопрос,Ссылка на видео,Ссылка на вопрос в FAQ,Где используется на сайте,Шаг 1,Шаг 2,Шаг 3,Шаг 4,Шаг 5,Шаг 6,Шаг 7,Шаг 8,Шаг 9,Шаг 10,Шаг 11,Шаг 12,Шаг 13,Шаг 14,Вопрос_1
0,Карты,Активация,ИБ,Как активировать карту,NaN,NaN,https://www.open.ru/credit_cards/faq#1056\n \n...,На вкладке «Мои продукты» в разделе «Карты» вы...,Введите предпоследние 4 цифры номера карты — с...,Придумайте и повторно введите PIN-код. Он не д...,Нажмите «Активировать»,Введите SMS-код для подтверждения операции,Ваша карта активирована,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,активировать карта
1,Карты,Баланс,ИБ,Как узнать баланс карты,https://www.youtube.com/watch?v=M-C95qRN15Y&fe...,NaN,https://www.open.ru/faq#1096\n \n https://www....,На вкладке «Мои продукты» в разделе «Карты» вы...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,узнать баланс карта
2,Карты,История операций,ИБ,Как посмотреть историю операций по карте,NaN,NaN,NaN,Перейдите в раздел «История операций». На экра...,Выберите нужную карту,"Выберите период, тип операций и категории",Ниже появятся операции по карте по выбранным в...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,посмотреть история операция карта
3,Карты,Справки и выписки,ИБ,Как получить выписку по карте с синей печатью ...,https://www.youtube.com/watch?v=M-C95qRN15Y&fe...,NaN,NaN,Перейдите в раздел «История операций». На экра...,Выберите нужную карту,"Выберите период, тип (»Все», «Пополнения» или ...",Выберите формат выписки в поле «Получить выпис...,Выписку в формате PDF вы можете скачать или от...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,получить выписка карта синь печать банк факсимиле
4,Карты,Реквизиты,ИБ,Как посмотреть реквизиты счета карты,NaN,NaN,NaN,На вкладке «Мои продукты» найдите нужную карту...,Перейдите в раздел «Счет»,В открывшемся окне отобразятся реквизиты счета...,Реквизиты можно отправить на e-mail. Для этого...,ОЙ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,посмотреть реквизит счёт карта
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Накопительные счета,Повышение ставки,ИБ,Как повысить ставку по накопительному счету «М...,NaN,https://www.open.ru/deposits/savings_account/f...,https://www.open.ru/deposits/savings_account/faq,Перейдите на экран программы лояльности МАКС: ...,Нажмите «Потратить бонусы».,Нажмите «Повысить ставку по счету».,"Выберите накопительный счет, для которого хоти...","Выберите, на какой срок повысить ставку: на од...",Перетяните подходящий вариант повышения ставки...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,повысить ставка накопительный счёт « копилка »...
285,Карты,Аналитика финансов,ИБ,Как посмотреть аналитику финансов по карте,NaN,https://www.open.ru/faq#1829,https://www.open.ru/faq,Перейдите на вкладку «История операций».,Выберите карту или несколько карт в одной валю...,Диаграмма с аналитикой финансов отобразится в ...,Аналитику финансов можно просматривать в разны...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,посмотреть аналитик финансы карта
286,Календарь платежей,NaN,ИБ,Как посмотреть календарь платежей,NaN,NaN,NaN,Перейдите на вкладку «Мои продукты» или «Плате...,Перейдите в раздел «Календарь платежей» и нажм...,Справа от календаря отобразится список всех пл...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,посмотреть календарь платёж
287,Виртуальная карта,Дополнительная карта,ИБ,Как оформить дополнительную виртуальную карту,NaN,NaN,NaN,На экране «Мои продукты» в разделе «Карты» выб...,Выберите «Дополнительная виртуальная карта».,"Выберите платежную систему, ознакомьтесь с тар...",Введите SMS-код для подтверждения выпуска карты.,Карта будет выпущена через несколько минут и п...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,оформить дополнительный виртуальный карта
